In [1]:
import pandas as pd
import warnings
import re
import tables
from datetime import datetime
warnings.filterwarnings('ignore')

# Funcao que formata tabela no modelo da genealogia embrapa

In [2]:
def formata_df_no_modelo_genealogia_embrapa(df):
    #trata o dnasc
    if 'DNASC' in df.columns:
        df = df.rename(columns={'DNASC':'nasc'})
    #trata o formata_df_no_modelo_genealogia_embrapa
    if 'NOMEVACA' in df.columns:
        df = df.rename(columns={'NOMEVACA':'nome'})
    # Adiciona coluna "id"
    df = df.assign(id=None)
    #Atribui valores de id_produto a id
    df['id'] = df['id_produto']
    # Adiciona coluna "abcz" com valores iguais a null
    df = df.assign(abcz=None)
    #renomeia RGVACA para registro
    df = df.rename(columns={'RGVACA':'registro'})
    #renomeia nomea para nome
    df = df.rename(columns={'nomea':'nome'})
    #renomeia SEXO para sexo
    df = df.rename(columns={'SEXO':'sexo'})
    # Adiciona coluna "paternidade" com valores iguais a null
    df = df.assign(paternidade=None)
    # Adiciona coluna "ck_sire" com valores iguais a null
    df = df.assign(ck_sire=None)
    # Adiciona coluna "ck_dam" com valores iguais a null
    df = df.assign(ck_dam=None)
    # Adiciona coluna "gen_valido" com valores iguais a null
    df = df.assign(gen_valido=None)
    # Adiciona coluna "nasc_est"
    df = df.assign(nasc_est=None)
    #Atribui os valores de nasc a nasc_est
    df['nasc_est'] = df['nasc']
    # Adiciona coluna "gr_psg"
    df = df.assign(gr_psg=None)
    # Adiciona coluna "gr_ps"
    df = df.assign(gr_ps=None)
    # Adiciona coluna "gr_tp"
    df = df.assign(gr_tp=None)
    # Adiciona coluna "rebc2_cri"
    df = df.assign(rebc_cri=None)
    #renomeia rebc_or para rebc_pr
    df = df.rename(columns={'rebc_or':'rebc_pr'})
    # Adiciona coluna "cat"
    if 'cat' in df.columns:
        pass
    else:
        df = df.assign(cat=None)
    # Adiciona coluna "ficha_a"
    df = df.assign(ficha_a=None)
    #renomeia IDANIMAL para idanimal
    df = df.rename(columns={'IDANIMAL':'idanimal'})
    # Adiciona coluna "origem"
    df = df.assign(origem='GIRO')
    # Adiciona coluna "data_insercao"
    df = df.assign(data_insercao=None)
    data_atual = datetime.today().strftime('%Y-%m-%d')
    df['data_insercao'] = data_atual
    df = df[['id_produto', 'id', 'abcz', 'registro','nome','sexo','paternidade','ck_sire','id_touro','id_vaca','ck_dam','gen_valido','nasc','nasc_est','cgen_a','gs','gr_psg','gr_ps','gr_tp','rebc_cri','rebc_pr','cat','ficha_a','cod_prod','idanimal','origem', 'data_insercao']]
    return df


In [ ]:
def busca_id_genealogia_embrapa(df):
    df = pd.merge(df, df[['id_produto', 'IDANIMAL']],left_on=['IDPAI'], how='left', right_on=['IDANIMAL'], suffixes=('', '_drop'))
    df = df.drop_duplicates()
    df = pd.merge(df, df_genealogia_embrapa[['id_produto', 'IDANIMAL']],left_on=['IDMAE'], how='left', right_on=['IDANIMAL'], suffixes=('', '_tes'))
    df = df.drop_duplicates()
    df = df.rename(columns={'id_produto_drop':'id_touro'})
    df = df.rename(columns={'id_produto_tes':'id_vaca'})
    df = df.drop(columns='IDANIMAL_drop')
    df = df.drop(columns='IDANIMAL_tes')
    return df

In [ ]:
def busca_id_touro_e_id_vaca_pelo_cod_prod_genealogia_embrapa(df):
    df_id_touro = pd.merge(df, df_genealogia_embrapa[['id_produto', 'cod_prod']], left_on=['cod_pai'], right_on=['cod_prod'], suffixes=('', '_drop'))
    #ID_TOURO
    df = pd.merge(df, df_id_touro[['IDANIMAL','id_produto_drop']], on=['IDANIMAL'], how='left')
    df = df.drop_duplicates()
    df['id_touro'].fillna(df['id_produto_drop'], inplace=True)
    df = df.drop(columns='id_produto_drop')

    #ID_VACA
    df_id_vaca = pd.merge(df, df_genealogia_embrapa[['id_produto', 'cod_prod']], left_on=['cod_mae'], right_on=['cod_prod'], indicator=True, suffixes=('', '_drop'))
    df = pd.merge(df, df_id_vaca[['IDANIMAL','id_produto_drop']], on=['IDANIMAL'], how='left')
    df = df.drop_duplicates()
    df['id_vaca'].fillna(df['id_produto_drop'], inplace=True)
    df = df.drop(columns='id_produto_drop')
    return df

In [ ]:
def pega_id_produto_genealogia_embrapa_registro_3_letras_e_GSPAI_GSMAE_11(df):
    df_sub_tabela_gir = df[df['GSPAI']==11]
    df_sub_tabela_gir = df[df['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
    df_sub_tabela_gir1 = df[df['GSMAE']==11]
    df_sub_tabela_gir1 = df[df['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
    df_sub_touros_id = pd.merge(df_sub_tabela_gir, df_genealogia_embrapa[['id_produto','registro']], left_on=['RGPAI'], right_on=['registro'])
    df_sub_vacas_id = pd.merge(df_sub_tabela_gir1, df_genealogia_embrapa[['id_produto','registro']], left_on=['RGMAE'], right_on=['registro'])

    df = pd.merge(df, df_sub_touros_id[['IDANIMAL', 'id_produto_y']], on=['IDANIMAL'], how='left')
    df['id_touro'].fillna(df['id_produto_y'], inplace=True)
    df = df.drop(columns='id_produto_y')
    df = pd.merge(df, df_sub_vacas_id[['IDANIMAL', 'id_produto_y']], on=['IDANIMAL'], how='left')
    df['id_vaca'].fillna(df['id_produto_y'], inplace=True)
    df = df.drop(columns='id_produto_y')
    return df

In [ ]:
def pega_id_produto_touros_vacas_genealogia_RG_menor_3(df):
    df_sub_tabela_gir1 = df[df['GSPAI']==11]
    df_sub_tabela_gir1 = df_sub_tabela_gir1[~df_sub_tabela_gir1['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
    #Pegando os 4 primeiros caracteres do nome
    df_sub_tabela_gir1['nome4caracteres'] = df_sub_tabela_gir1['nomep'].str.slice(start=0, stop=4)


    df_subtabela_genealogia = df_genealogia_embrapa
    df_subtabela_genealogia = df_subtabela_genealogia[df_subtabela_genealogia['gs']==0]
    df_subtabela_genealogia = df_subtabela_genealogia[df_subtabela_genealogia['sexo']=='M']
    df_subtabela_genealogia = df_subtabela_genealogia[~df_subtabela_genealogia['registro'].isnull()]
    df_subtabela_genealogia = df_subtabela_genealogia[~df_subtabela_genealogia['nome'].isnull()]
    df_subtabela_genealogia = df_subtabela_genealogia[~df_subtabela_genealogia['registro'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
    df_subtabela_genealogia['nome4caracteres'] = df_subtabela_genealogia['nome'].str.slice(start=0, stop=4)

    dj_juntando_tabelas = pd.merge(df_sub_tabela_gir1, df_subtabela_genealogia[['id_produto', 'registro', 'nome4caracteres']], left_on=['nome4caracteres', 'RGPAI'], right_on=['nome4caracteres', 'registro'])

    df_sub_tabela_gir2 = df[df['GSMAE']==11]
    df_sub_tabela_gir2 = df_sub_tabela_gir2[~df_sub_tabela_gir2['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
    #Pegando os 4 primeiros caracteres do nome
    df_sub_tabela_gir2['nome4caracteres'] = df_sub_tabela_gir2['nomem'].str.slice(start=0, stop=4)


    df_subtabela_genealogia1 = df_genealogia_embrapa
    df_subtabela_genealogia1 = df_subtabela_genealogia1[df_subtabela_genealogia1['gs']==0]
    df_subtabela_genealogia1 = df_subtabela_genealogia1[df_subtabela_genealogia1['sexo']=='F']
    df_subtabela_genealogia1 = df_subtabela_genealogia1[~df_subtabela_genealogia1['registro'].isnull()]
    df_subtabela_genealogia1 = df_subtabela_genealogia1[~df_subtabela_genealogia1['nome'].isnull()]
    df_subtabela_genealogia1 = df_subtabela_genealogia1[~df_subtabela_genealogia1['registro'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
    df_subtabela_genealogia1['nome4caracteres'] = df_subtabela_genealogia1['nome'].str.slice(start=0, stop=4)

    dj_juntando_tabelas1 = pd.merge(df_sub_tabela_gir2, df_subtabela_genealogia1[['id_produto', 'registro', 'nome4caracteres']], left_on=['nome4caracteres', 'RGMAE'], right_on=['nome4caracteres', 'registro'])

    df = pd.merge(df, dj_juntando_tabelas[['IDANIMAL', 'id_produto_y']], on=['IDANIMAL'], how='left')
    df['id_touro'].fillna(df['id_produto_y'], inplace=True)
    df = df.drop(columns='id_produto_y')
    df = pd.merge(df, dj_juntando_tabelas1[['IDANIMAL', 'id_produto_y']], on=['IDANIMAL'], how='left')
    df['id_vaca'].fillna(df['id_produto_y'], inplace=True)
    df = df.drop(columns='id_produto_y')

# Lendo planilhas

In [3]:
df_animais_sem_id = pd.read_hdf('df_animais_sem_id.h5', key='df')
df_genealogia_embrapa = pd.read_hdf('df_genealogia_embrapa.h5', key='df')
df_fazendas = pd.read_hdf('df_fazendas.h5', key='df')
df_criadores = pd.read_hdf('df_criadores.h5', key='df')
df_genealogia_abcz = pd.read_hdf('df_genealogia_abcz.h5', key='df')
pedigree = pd.read_csv("pedrigree - Girolando - 17-02-2023.txt", encoding='ISO-8859-1')

# Imprimindo dataframes

In [4]:
df_animais_sem_id = df_animais_sem_id.drop_duplicates()
df_animais_sem_id = df_animais_sem_id.drop(columns='id_produto_x')
df_animais_sem_id = df_animais_sem_id.drop(columns='id_produto_y')
#df_animais_sem_id

# Criando dataframe df_animais_sem_id_completo pela juncao do df_animais_sem_id com pedigree
Obs: Mantendo somente o primeiro registro dos animais com IDANIMAL duplicados

In [5]:
df_animais_sem_id_completo = pd.merge(df_animais_sem_id, pedigree, on=['IDANIMAL'], how='left', indicator=True, suffixes=('','_drop'))
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(df_animais_sem_id_completo.filter(regex='_drop$').columns, axis=1)
df_animais_sem_id_completo = df_animais_sem_id_completo[df_animais_sem_id_completo['_merge']=='both']
df_animais_sem_id_completo = df_animais_sem_id_completo.drop_duplicates(subset='IDANIMAL')
#df_animais_sem_id_completo

# Criando id_produto dos animais que não possuem id_produto
obs: id_produto = giro+IDANIMAL

In [6]:
df_animais_sem_id_completo['id_produto'] = df_animais_sem_id_completo['id_produto'].fillna('GIRO' + df_animais_sem_id_completo['IDANIMAL'].astype(str))
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='_merge')
#df_animais_sem_id_completo

# Juntando df_animais_sem_id_completo (idpai) com genealogia_embrapa (idanimal) tabela separada para pedigree

In [7]:
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, df_genealogia_embrapa[['id_produto', 'IDANIMAL']],left_on=['IDPAI'], how='left', right_on=['IDANIMAL'], suffixes=('', '_drop'))
df_animais_sem_id_completo = df_animais_sem_id_completo.drop_duplicates()
#df_animais_sem_id_completo

# Juntando df_animais_sem_id_completo (idmae) com genealogia_embrapa (idanimal)

In [8]:
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, df_genealogia_embrapa[['id_produto', 'IDANIMAL']],left_on=['IDMAE'], how='left', right_on=['IDANIMAL'], suffixes=('', '_tes'))
df_animais_sem_id_completo = df_animais_sem_id_completo.drop_duplicates()
#df_animais_sem_id_completo

# Adicionando coluna IDTOURO e id_vaca no df_animais_sem_id_completo

In [9]:
df_animais_sem_id_completo = df_animais_sem_id_completo.rename(columns={'id_produto_drop':'id_touro'})
df_animais_sem_id_completo = df_animais_sem_id_completo.rename(columns={'id_produto_tes':'id_vaca'})
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='IDANIMAL_drop')
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='IDANIMAL_tes')
#df_animais_sem_id_completo

# Buscando o id_touro pelo cod_pai na genealogia_embrapa

In [10]:
df_id_touro = pd.merge(df_animais_sem_id_completo, df_genealogia_embrapa[['id_produto', 'cod_prod']], left_on=['cod_pai'], right_on=['cod_prod'], suffixes=('', '_drop'))
#df_id_touro

# Adicionando o id_touro no dataframe com todas as colunas pelo id_animal

In [11]:
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, df_id_touro[['IDANIMAL','id_produto_drop']], on=['IDANIMAL'], how='left')
df_animais_sem_id_completo = df_animais_sem_id_completo.drop_duplicates()
df_animais_sem_id_completo['id_touro'].fillna(df_animais_sem_id_completo['id_produto_drop'], inplace=True)
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='id_produto_drop')
#df_animais_sem_id_completo

# Buscando o id_vaca pelo cod_vaca na genealogia_embrapa

In [12]:
df_id_vaca = pd.merge(df_animais_sem_id_completo, df_genealogia_embrapa[['id_produto', 'cod_prod']], left_on=['cod_mae'], right_on=['cod_prod'], indicator=True, suffixes=('', '_drop'))
#df_id_vaca

# Adicionando id_vaca no dataframe com todas as colunas

In [13]:
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, df_id_vaca[['IDANIMAL','id_produto_drop']], on=['IDANIMAL'], how='left')
df_animais_sem_id_completo = df_animais_sem_id_completo.drop_duplicates()
df_animais_sem_id_completo['id_vaca'].fillna(df_animais_sem_id_completo['id_produto_drop'], inplace=True)
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='id_produto_drop')
#df_animais_sem_id_completo

# Pegando o id_produto que vira id_touro dos touros_gir na genealogia embrapa, em que o registro comeca com 3 letras

In [14]:
# somente gs_pai=11 e gs = 0 na genealogia
df_sub_tabela_gir = df_animais_sem_id_completo[df_animais_sem_id_completo['GSPAI']==11]
df_sub_tabela_gir = df_sub_tabela_gir[df_sub_tabela_gir['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
#df_sub_tabela_gir

# Pegando o id_produto que vira id_vaca das vacas_gir na genealogia embrapa, em que o registro comeca com 3 letras

In [15]:
# somente gs_pai=11 e gs = 0 na genealogia
df_sub_tabela_gir1 = df_animais_sem_id_completo[df_animais_sem_id_completo['GSMAE']==11]
df_sub_tabela_gir1 = df_sub_tabela_gir1[df_sub_tabela_gir1['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
#df_sub_tabela_gir1

# Juntando a genealogia embrapa com os touros comecando com 3 letras e gs 11

In [16]:
df_sub_touros_id = pd.merge(df_sub_tabela_gir, df_genealogia_embrapa[['id_produto','registro']], left_on=['RGPAI'], right_on=['registro'])
#df_sub_touros_id

# Juntando a genealogia embrapa com as vacas comecando com 3 letras e gs 11

In [17]:
df_sub_vacas_id = pd.merge(df_sub_tabela_gir1, df_genealogia_embrapa[['id_produto','registro']], left_on=['RGMAE'], right_on=['registro'])
#df_sub_vacas_id

In [18]:
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, df_sub_touros_id[['IDANIMAL', 'id_produto_y']], on=['IDANIMAL'], how='left')
df_animais_sem_id_completo['id_touro'].fillna(df_animais_sem_id_completo['id_produto_y'], inplace=True)
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='id_produto_y')
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, df_sub_vacas_id[['IDANIMAL', 'id_produto_y']], on=['IDANIMAL'], how='left')
df_animais_sem_id_completo['id_vaca'].fillna(df_animais_sem_id_completo['id_produto_y'], inplace=True)
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='id_produto_y')
#df_animais_sem_id_completo

Juntar codigo acima com genealogia
RGPAI(df_sub_tabela_gir) liga com registro(genealogia)
duplicar solucao para mae
RGPAI RGMAE RGVACA GS=11 retirar acentos e espacos

# Pegando os id_produto's dos touros_gir na genealogia embrapa, em que o registro nao comeca com 3 letras

In [19]:
df_sub_tabela_gir1 = df_animais_sem_id_completo[df_animais_sem_id_completo['GSPAI']==11]
df_sub_tabela_gir1 = df_sub_tabela_gir1[~df_sub_tabela_gir1['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
#Pegando os 4 primeiros caracteres do nome
df_sub_tabela_gir1['nome4caracteres'] = df_sub_tabela_gir1['nomep'].str.slice(start=0, stop=4)


df_subtabela_genealogia = df_genealogia_embrapa
df_subtabela_genealogia = df_subtabela_genealogia[df_subtabela_genealogia['gs']==0]
df_subtabela_genealogia = df_subtabela_genealogia[df_subtabela_genealogia['sexo']=='M']
df_subtabela_genealogia = df_subtabela_genealogia[~df_subtabela_genealogia['registro'].isnull()]
df_subtabela_genealogia = df_subtabela_genealogia[~df_subtabela_genealogia['nome'].isnull()]
df_subtabela_genealogia = df_subtabela_genealogia[~df_subtabela_genealogia['registro'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
df_subtabela_genealogia['nome4caracteres'] = df_subtabela_genealogia['nome'].str.slice(start=0, stop=4)

dj_juntando_tabelas = pd.merge(df_sub_tabela_gir1, df_subtabela_genealogia[['id_produto', 'registro', 'nome4caracteres']], left_on=['nome4caracteres', 'RGPAI'], right_on=['nome4caracteres', 'registro'])
#dj_juntando_tabelas

# Pegando os id_produto’s das vacas na genealogia embrapa, em que o registro nao comeca com 3 letras

In [20]:
df_sub_tabela_gir2 = df_animais_sem_id_completo[df_animais_sem_id_completo['GSMAE']==11]
df_sub_tabela_gir2 = df_sub_tabela_gir2[~df_sub_tabela_gir2['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
#Pegando os 4 primeiros caracteres do nome
df_sub_tabela_gir2['nome4caracteres'] = df_sub_tabela_gir2['nomem'].str.slice(start=0, stop=4)


df_subtabela_genealogia1 = df_genealogia_embrapa
df_subtabela_genealogia1 = df_subtabela_genealogia1[df_subtabela_genealogia1['gs']==0]
df_subtabela_genealogia1 = df_subtabela_genealogia1[df_subtabela_genealogia1['sexo']=='F']
df_subtabela_genealogia1 = df_subtabela_genealogia1[~df_subtabela_genealogia1['registro'].isnull()]
df_subtabela_genealogia1 = df_subtabela_genealogia1[~df_subtabela_genealogia1['nome'].isnull()]
df_subtabela_genealogia1 = df_subtabela_genealogia1[~df_subtabela_genealogia1['registro'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
df_subtabela_genealogia1['nome4caracteres'] = df_subtabela_genealogia1['nome'].str.slice(start=0, stop=4)

dj_juntando_tabelas1 = pd.merge(df_sub_tabela_gir2, df_subtabela_genealogia1[['id_produto', 'registro', 'nome4caracteres']], left_on=['nome4caracteres', 'RGMAE'], right_on=['nome4caracteres', 'registro'])
#dj_juntando_tabelas1

# Adicionando ID's touros e ID's vacas encontrados em que o registro não comeca com 3 letras na df_animais_sem_id_completo

In [21]:
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, dj_juntando_tabelas[['IDANIMAL', 'id_produto_y']], on=['IDANIMAL'], how='left')
df_animais_sem_id_completo['id_touro'].fillna(df_animais_sem_id_completo['id_produto_y'], inplace=True)
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='id_produto_y')
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, dj_juntando_tabelas1[['IDANIMAL', 'id_produto_y']], on=['IDANIMAL'], how='left')
df_animais_sem_id_completo['id_vaca'].fillna(df_animais_sem_id_completo['id_produto_y'], inplace=True)
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='id_produto_y')
#df_animais_sem_id_completo

# Lista de animais com id_vaca E id_touro

In [22]:
df_animais_com_ids = df_animais_sem_id_completo[~df_animais_sem_id_completo['id_vaca'].isnull() & ~df_animais_sem_id_completo['id_touro'].isnull()]
#df_animais_com_ids

# Lista de animais sem id_vaca

In [23]:
df_animais_sem_id_vaca = df_animais_sem_id_completo[df_animais_sem_id_completo['id_vaca'].isnull()]
df_animais_sem_id_vaca = df_animais_sem_id_vaca.drop_duplicates()
#df_animais_sem_id_vaca

# Lista de animais sem id_touro

In [24]:
df_animais_sem_id_touro = df_animais_sem_id_completo[df_animais_sem_id_completo['id_touro'].isnull()]
df_animais_sem_id_touro = df_animais_sem_id_touro.drop_duplicates()
#df_animais_sem_id_touro

# Lista de animais sem ID

In [25]:
df_animais_sem_id = pd.concat([df_animais_sem_id_touro, df_animais_sem_id_vaca])
#df_animais_sem_id

# Juntando animais sem ID com o Pedigree

In [28]:
df_animais_sem_id['id_touro'] = df_animais_sem_id['id_touro'].fillna('GIRO' + df_animais_sem_id['IDPAI'].astype(str))
df_animais_sem_id['id_vaca'] = df_animais_sem_id['id_vaca'].fillna('GIRO' + df_animais_sem_id['IDMAE'].astype(str))

df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, df_genealogia_embrapa[['id_produto', 'IDANIMAL']],left_on=['IDPAI'], how='left', right_on=['IDANIMAL'], suffixes=('', '_drop'))
df_animais_sem_id_completo = df_animais_sem_id_completo.drop_duplicates()
#df_animais_sem_id_completo



df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, df_genealogia_embrapa[['id_produto', 'IDANIMAL']],left_on=['IDMAE'], how='left', right_on=['IDANIMAL'], suffixes=('', '_tes'))
df_animais_sem_id_completo = df_animais_sem_id_completo.drop_duplicates()
#df_animais_sem_id_completo




df_juntando_teste = pd.merge(df_animais_sem_id['IDPAI'], pedigree, left_on=['IDPAI'], right_on=['IDANIMAL'], suffixes=['_drop', ''])
df_juntando_teste = df_juntando_teste.drop_duplicates()
df_juntando_teste1 = pd.merge(df_animais_sem_id['IDMAE'], pedigree, left_on=['IDMAE'], right_on=['IDANIMAL'], suffixes=['_drop', ''])
df_juntando_teste1 = df_juntando_teste1.drop_duplicates()
df_juntando_teste3 = pd.concat([df_juntando_teste1, df_juntando_teste])
#Criando coluna id_produto= none e preenchendo id_produto
df_juntando_teste3 = df_juntando_teste3.assign(id_produto=None)
df_juntando_teste3['id_produto'] = df_juntando_teste3['id_produto'].fillna('GIRO' + df_juntando_teste3['IDPAI_drop'].astype(str))
df_juntando_teste3['id_produto'] = df_juntando_teste3['id_produto'].fillna('GIRO' + df_juntando_teste3['IDMAE_drop'].astype(str))
#Criando coluna id_touro= none
df_juntando_teste3 = df_juntando_teste3.assign(id_touro=None)
df_juntando_teste3 = df_juntando_teste3.assign(id_touro=None)
df_juntando_teste3['IDPAI'] = df_juntando_teste3['IDPAI'].fillna('')
df_juntando_teste3['id_touro'] = None
for i in range(len(df_juntando_teste3)):
    if df_juntando_teste3['IDPAI'].iloc[i] != '':
        df_juntando_teste3['id_touro'].iloc[i] = 'GIRO' + str(df_juntando_teste3['IDPAI'].iloc[i])
#Criando coluna id_vaca= none
df_juntando_teste3 = df_juntando_teste3.assign(id_vaca=None)
df_juntando_teste3['IDMAE'] = df_juntando_teste3['IDMAE'].fillna('')
df_juntando_teste3['id_vaca'] = None
for i in range(len(df_juntando_teste3)):
    if df_juntando_teste3['IDMAE'].iloc[i] != '':
        df_juntando_teste3['id_vaca'].iloc[i] = 'GIRO' + str(df_juntando_teste3['IDMAE'].iloc[i])


# Pegando rebc2_pr da tabela de fazendas
df_juntando_teste3 = pd.merge(df_juntando_teste3, df_criadores[['rebc_or', 'codgiro']], left_on=['reb_giro'], right_on=['codgiro'])
df_animais_com_ids = df_animais_com_ids.drop_duplicates()





df_juntando_teste3 = df_juntando_teste3.drop(df_juntando_teste3.filter(regex='_drop$').columns, axis=1)
df_juntando_teste3

,NOMEVACA,RGVACA,COMPRAC,IDANIMAL,DNASC,SEXO,NOMEPAI,RGPAI,GSPAI,IDPAI,NOMEMAE,RGMAE,GSMAE,IDMAE,gs,cgen_a,comb,id_produto,id_touro,id_vaca
0,SALINA NOBRE DO MORRO,C-4972,14,821357.0,26/09/2008,F,NOBRE TE CAL,CAL4397,11.0,3142.0,OLIMPIADA DJ DO MORRO,X-6776,12.0,3142.0,2,1/4HOL 3/4GIR,315,GIROnan,GIRO3142.0,GIRO3142.0
1,DIAMANTINA FIV ZBR,ZBR157,11,1230680.0,10/10/2008,F,TEATRO DA SILVANIA,EFC383,11.0,5024.0,QUICA DA CAL,CAL5081,11.0,5024.0,0,GIR,11,GIROnan,GIRO5024.0,GIRO5024.0
2,UTOPIA FIV 2B,ZAB1116,11,2357203.0,26/03/2015,F,TABU TE DA CAL,CAL6557,11.0,877840.0,INCISAO FIV DE BRAS.,RRP6711,11.0,877840.0,0,GIR,11,GIROnan,GIRO877840.0,GIRO877840.0
3,PRATEADA FIV SANSAO JGVA,3599-F,14,984858.0,05/10/2009,F,C.A.SANSAO,KCA472,11.0,3234.0,PATATIVA MARKOWICZ,U-1676,12.0,3234.0,2,1/4HOL 3/4GIR,315,GIROnan,GIRO3234.0,GIRO3234.0
4,BERLINDA JAYVEN JM NOVO HORIZONTE,5203-AE,12,1388188.0,03/06/2013,F,STOUDER JAYVEN-ET,AX134762,10.0,1104837.0,P.L. ROSADA MARAJA,PNL187,11.0,1104837.0,4,1/2HOL 1/2GIR,519,GIROnan,GIRO1104837.0,GIRO1104837.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,STANTONS CAPITAL GAIN,AX146382,10,1846259.0,13/02/2013,M,DE-SU BKM MCCUTCHEN 1174-ET,AX140261,10.0,1277724.0,MS CHASSITY OBS CLAIRE-ET,US141008075,10.0,1277724.0,8,HOL,999,GIRO1846259.0,GIRO1277724.0,GIRO1277724.0
215,POL BUTTE MC BEEMER-ET,AX145127,10,1606782.0,11/04/2013,M,DE-SU BKM MCCUTCHEN 1174-ET,AX140261,10.0,1277724.0,BUTZ-BUTLER GOLD BANNER-ET,US140145517,10.0,1277724.0,8,HOL,999,GIRO1606782.0,GIRO1277724.0,GIRO1277724.0
216,GALEGO FIV F. MUTUM,MUT899,11,1489363.0,08/08/2007,M,C.A.SANSAO,KCA472,11.0,3234.0,URGENCIA 3R B.MONTE,LAC123,11.0,3234.0,0,GIR,11,GIRO1489363.0,GIRO3234.0,GIRO3234.0
217,FEBO FR RECREIO,6300-N,58,1114289.0,20/12/2011,M,STANBRO MORE,AX118094,10.0,991986.0,LAPAZ FR RECREIO,B-0054,14.0,991986.0,5,5/8HOL 3/8GIR,693,GIRO1114289.0,GIRO991986.0,GIRO991986.0


In [ ]:
# Cirnao

In [30]:
test = formata_df_no_modelo_genealogia_embrapa(df_juntando_teste3)
test

KeyError: "['nome', 'id_touro', 'id_vaca', 'rebc_pr', 'cod_prod'] not in index"

# IDPAI != Nan

In [ ]:
df_juntando_teste1 = df_juntando_teste
#Transformando id_touro em id_produto no dataframe
df_juntando_teste1['id_produto_pedigree'] = df_juntando_teste1['id_touro_original']
#Zerando o id_touro_original
df_juntando_teste1['id_touro_original'] = None
#Zerando o id_vaca_original
df_juntando_teste1['id_vaca_original'] = None
#Renomeando id_touro_original para id_touro
df_juntando_teste1 = df_juntando_teste1.rename(columns={'id_touro_original':'id_vaca'})
df_juntando_teste1








#df_juntando_teste1 = df_juntando_teste.drop(df_juntando_teste.filter(regex='_original$').columns, axis=1)


Lista abaixo juntar ID_PAI com id_animal no pedigree
drop duplicate id_animal
criar id_touro nessa tabela sendo GIRO+IDPAI (quando idPAI != nan)
id_touro vai virar id_produto na df_animais_com_ids
id_pai e id_mae nulo pausa a recursividade

# Juntar df_animais_sem_id_touro pelo id_pai com id_animal no pedigree
GSPAI != 0

In [ ]:
df_animais_sem_id_touro['id_touro'] = df_animais_sem_id_touro['id_touro'].fillna('GIRO' + df_animais_sem_id_touro['IDPAI'].astype(str))
df_teste = pd.merge(df_animais_sem_id_touro, pedigree, left_on=['IDPAI'], right_on=['IDANIMAL'], suffixes=('_dir', '_esq'))
#df_teste = df_teste.drop(df_teste.filter(regex='_drop$').columns, axis=1)
df_teste = df_teste.drop_duplicates()
#df_teste = df_teste.drop_duplicates(subset='IDANIMAL')
#Criando id_touro
df_teste['id_produto'] = df_teste['id_touro'].fillna('GIRO' + df_teste['IDPAI'].astype(str))
df_teste

Tabela de fazendas pegar o rebc2_pr -> rebc2-or (fazendas) <- reb_giro(df_animais_com_id)

ABCZ - S = Quando cod prod > 0 ABCZ
cat cod_prod >0 (5 comprac 11) ABCZ

id = id_produto
nas e nasc_est = DNASC


# Separando os animais com id_touro e id_vaca que possuem COMPRAC = 11 dos que nao possuem

In [ ]:
# Pegando rebc2_pr da tabela de fazendas
df_animais_com_ids = pd.merge(df_animais_com_ids, df_criadores[['rebc_or', 'codgiro']], left_on=['reb_giro'], right_on=['codgiro'])
df_animais_com_ids = df_animais_com_ids.drop_duplicates()
df_animais_com_ids_abcz = df_animais_com_ids[df_animais_com_ids['COMPRAC']==11]
df_animais_com_ids = df_animais_com_ids[df_animais_com_ids['COMPRAC']!=11]
df_animais_com_ids

# Tabela com os animais que possuem comprac = 11

In [ ]:
df_animais_com_ids_abcz

# Juntando a tabela dos animais com ids encontrados que possuem comprac = 11 com o genealogia abcz

In [ ]:
df_animais_com_ids_abcz = pd.merge(df_animais_com_ids_abcz, df_genealogia_abcz[['cod_prod', 'cat']], on=['cod_prod'])
df_animais_com_ids_abcz

# Formatando dataframes no modelo da tabela Genealogia Embrapa do Banco de dados

In [ ]:
df_animais_com_ids_abcz = formata_df_no_modelo_genealogia_embrapa(df_animais_com_ids_abcz)
df_animais_com_ids = formata_df_no_modelo_genealogia_embrapa(df_animais_com_ids)
df_animais_com_ids = pd.concat([df_animais_com_ids, df_animais_com_ids_abcz])
df_animais_com_ids